In [1]:
import pandas as pd
import yaml

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.2f}'.format)


with open('../../Settings.yaml', 'r') as file:
    Setting = yaml.safe_load(file)

file_name = "Capital_Stock.xlsx"
file_path = f"{Setting['Raw_Path']}/{file_name}"
Capital_Stock = pd.read_excel(file_path, sheet_name = 1 , skiprows=2)

Capital_Stock.rename(columns={'فعاليت / سال':'Category'},inplace=True)

Capital_Stock = pd.melt(Capital_Stock, id_vars='Category', var_name='Year', value_name='Amount')

#industry capital amount at year 1381 in milion rial at current price
Industry_Capital_Amount = Capital_Stock[(Capital_Stock["Category"] == 'صنعت') & (Capital_Stock["Year"] == 1381)].Amount.iloc[0] * 1000

del Capital_Stock

In [2]:
file_name_workshop = "010_WorkShop_Statistics_By_Activity.xlsx"
file_path_workshop = f"{Setting['Output_Path_Unajusted']}/{file_name_workshop}"
workshop_statistics = pd.read_excel(file_path_workshop)

#creating new dataframe to show value added by sector at year 1381 by industry section
K0_Dataset = workshop_statistics[
    (workshop_statistics["Year"] == 1381) & 
    (workshop_statistics["Industry_Category_Code"] == 2)
][["Year","Industry_Code", "Total_Value_Added_Industrial_Activity"]]

#adding a new column to demonstrate how much each sector contributes. 
total_value_added = K0_Dataset["Total_Value_Added_Industrial_Activity"].sum()

K0_Dataset["Value_Added_Share"] =  K0_Dataset["Total_Value_Added_Industrial_Activity"] / total_value_added 

#estimating K0 for each sector by multiplying value_added_share to Capital_Formation.Fixed.Value
K0_Dataset["Capital"] = K0_Dataset["Value_Added_Share"] * Industry_Capital_Amount

K0_Dataset.drop(columns=["Total_Value_Added_Industrial_Activity","Value_Added_Share"],inplace=True)

#creating dataset which repersent investment amount of each sector at specific year
CapitalStock_Dataset = workshop_statistics[
    (workshop_statistics["Industry_Category_Code"] == 2)
][["Year","Industry_Code", "Capital_Formation.Fixed.Value"]]

CapitalStock_Dataset.rename(columns={"Capital_Formation.Fixed.Value":"Investment"},inplace=True)

#adding k0 from our previous dataset
CapitalStock_Dataset = pd.merge(
    CapitalStock_Dataset,
    K0_Dataset,
    how='left',
    on=['Year', 'Industry_Code']
)

#Assuming Depreciation_Rate is 5%
Depreciation_Rate = 0.05

# Sort the dataset by industry and year to ensure correct order
CapitalStock_Dataset.sort_values(by=["Industry_Code", "Year"], inplace=True)
CapitalStock_Dataset["Capital_Stock"] = None

# Set the initial capital for year 1381 (K0)
CapitalStock_Dataset.loc[CapitalStock_Dataset["Year"] == 1381, "Capital_Stock"] = \
    CapitalStock_Dataset.loc[CapitalStock_Dataset["Year"] == 1381, "Capital"]

# Loop over each industry and calculate capital stock for each year
for industry in CapitalStock_Dataset["Industry_Code"].unique():
    industry_df = CapitalStock_Dataset[CapitalStock_Dataset["Industry_Code"] == industry]
    industry_df = industry_df.sort_values("Year")

    for i in range(1, len(industry_df)):
        current_index = industry_df.index[i]
        prev_index = industry_df.index[i - 1]

        K_prev = CapitalStock_Dataset.loc[prev_index, "Capital_Stock"]
        I_t = CapitalStock_Dataset.loc[current_index, "Investment"]

        if pd.notnull(K_prev) and pd.notnull(I_t):
            K_t = (1 - Depreciation_Rate) * K_prev + I_t
            CapitalStock_Dataset.loc[current_index, "Capital_Stock"] = K_t

CapitalStock_Dataset.drop(columns="Capital",inplace=True)

output_file_name = '000_Capital_Stock.xlsx'
CapitalStock_Dataset.to_excel(f"{Setting['Output_Path_Ajusted']}/{output_file_name}", index=False)